<a href="https://www.kaggle.com/code/mohitheswar/wce-curated-colon-disease-detection-1?scriptVersionId=235353795" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Set up directories for training, validation, and test data
IMAGE_DIR = "/kaggle/input/curated-colon-dataset-for-deep-learning"
TRAIN_DIR = os.path.join(IMAGE_DIR, "train")
VAL_DIR = os.path.join(IMAGE_DIR, "val")
TEST_DIR = os.path.join(IMAGE_DIR, "test")

# List the class names
class_names = sorted(os.listdir(TRAIN_DIR))

# Image data generators with augmentation
train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=30, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Load data using generators
train_data = train_datagen.flow_from_directory(TRAIN_DIR, target_size=(224, 224), batch_size=32, class_mode='categorical')
val_data = val_test_datagen.flow_from_directory(VAL_DIR, target_size=(224, 224), batch_size=32, class_mode='categorical')
test_data = val_test_datagen.flow_from_directory(TEST_DIR, target_size=(224, 224), batch_size=32, class_mode='categorical')

# Callbacks for early stopping and learning rate reduction
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

# 🚀 Model Training & Evaluation Function
def train_and_evaluate_model(model_name, model_function):
    print(f"\nTraining {model_name}...")

    # Load the base model
    base_model = model_function(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    base_model.trainable = False

    # Construct the model
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(128, activation='relu'),
        layers.Dense(len(class_names), activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=0.0001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Train the model
    history = model.fit(
        train_data,
        validation_data=val_data,
        epochs=10,
        callbacks=[early_stop, lr_scheduler],
        verbose=1
    )

    # Evaluate on test data
    test_loss, test_accuracy = model.evaluate(test_data)

    print(f"\nModel: {model_name}")
    print(f"Training Accuracy: {max(history.history['accuracy'])*100:.2f}")
    print(f"Validation Accuracy: {max(history.history['val_accuracy'])*100:.2f}")
    print(f"Test Accuracy: {test_accuracy*100:.2f}, Test Loss: {test_loss*100:.2f}\n")

    return model

Found 3200 images belonging to 4 classes.
Found 2000 images belonging to 4 classes.
Found 800 images belonging to 4 classes.


In [2]:
train_and_evaluate_model("ResNet50", ResNet50)


Training ResNet50...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


100/100 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.2813 - loss: 1.4619 - val_accuracy: 0.2475 - val_loss: 1.3634 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 652ms/step - accuracy: 0.2935 - loss: 1.3676 - val_accuracy: 0.4165 - val_loss: 1.3409 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 652ms/step - accuracy: 0.3324 - loss: 1.3558 - val_accuracy: 0.4185 - val_loss: 1.3194 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 652ms/step - accuracy: 0.3982 - loss: 1.3399 - val_accuracy: 0.4520 - val_loss: 1.2993 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 664ms/step - accuracy: 0.4156 - loss: 1.3270 - val_accuracy: 0.5380 - val_loss: 1.2787 - learning_rate: 1.0000e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 645ms/step - accuracy: 0.4368 - loss: 1.3148 - val_accuracy: 0.4660 - val_loss: 1.2608 - learning_rate: 1.0000e-04
Epoch 7/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 672ms/step - ac

<Sequential name=sequential, built=True>

In [3]:
train_and_evaluate_model("VGG16", VGG16)


Training VGG16...
58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 104s 826ms/step - accuracy: 0.3183 - loss: 1.4004 - val_accuracy: 0.3005 - val_loss: 1.3136 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 686ms/step - accuracy: 0.6493 - loss: 1.2195 - val_accuracy: 0.3750 - val_loss: 1.2280 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 682ms/step - accuracy: 0.7554 - loss: 1.0707 - val_accuracy: 0.3985 - val_loss: 1.1751 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 682ms/step - accuracy: 0.7660 - loss: 0.9533 - val_accuracy: 0.4565 - val_loss: 1.1205 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 688ms/step - accuracy: 0.8002 - loss: 0.8505 - val_accuracy: 0.4855 - val_loss: 1.0879 - learning_rate: 1.0000e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 676ms/step - accuracy: 0.8057 - loss: 0.7823 - val_accuracy: 0.4325 - val_loss: 1.1184 - l

<Sequential name=sequential_1, built=True>

In [4]:
train_and_evaluate_model("InceptionV3", InceptionV3)


Training InceptionV3...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 96s 770ms/step - accuracy: 0.5934 - loss: 0.9874 - val_accuracy: 0.8730 - val_loss: 0.4229 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 658ms/step - accuracy: 0.8860 - loss: 0.3447 - val_accuracy: 0.8785 - val_loss: 0.3598 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 658ms/step - accuracy: 0.9129 - loss: 0.2457 - val_accuracy: 0.8860 - val_loss: 0.3236 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 657ms/step - accuracy: 0.9292 - loss: 0.2002 - val_accuracy: 0.8815 - val_loss: 0.3371 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 657ms/step - accuracy: 0.9336 - loss: 0.1963 - val_accuracy: 0.9220 - val_loss: 0.2326 - learning_rate: 1.0000e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 659ms/step - accuracy: 0.9503 - loss: 0.1653 - val_accuracy: 0.9160 - val_loss: 0.232

<Sequential name=sequential_2, built=True>

In [5]:
train_and_evaluate_model("DenseNet121", DenseNet121)


Training DenseNet121...
29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 115s 853ms/step - accuracy: 0.4428 - loss: 1.2390 - val_accuracy: 0.8035 - val_loss: 0.6015 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 660ms/step - accuracy: 0.8893 - loss: 0.4565 - val_accuracy: 0.8515 - val_loss: 0.4391 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 662ms/step - accuracy: 0.9211 - loss: 0.3067 - val_accuracy: 0.8995 - val_loss: 0.3120 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 663ms/step - accuracy: 0.9401 - loss: 0.2278 - val_accuracy: 0.9010 - val_loss: 0.2933 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 659ms/step - accuracy: 0.9497 - loss: 0.1842 - val_accuracy: 0.9190 - val_loss: 0.2383 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 258ms/step - accuracy: 0.8272 - loss: 0.5589

Model: DenseNet121
Training Accuracy: 95.22
Validati

<Sequential name=sequential_3, built=True>

In [6]:
train_and_evaluate_model("Xception", Xception)


Training Xception...
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 95s 751ms/step - accuracy: 0.6474 - loss: 0.9472 - val_accuracy: 0.8495 - val_loss: 0.5029 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 668ms/step - accuracy: 0.9178 - loss: 0.3054 - val_accuracy: 0.8915 - val_loss: 0.3494 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 674ms/step - accuracy: 0.9458 - loss: 0.2182 - val_accuracy: 0.8910 - val_loss: 0.3083 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 663ms/step - accuracy: 0.9422 - loss: 0.1944 - val_accuracy: 0.8980 - val_loss: 0.2876 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 668ms/step - accuracy: 0.9534 - loss: 0.1508 - val_accuracy: 0.9125 - val_loss: 0.2639 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 275ms/step - accuracy: 0.8616 - loss: 0.4944

Model: Xception
Training Accuracy: 95.47
Validation Accu

<Sequential name=sequential_4, built=True>

In [7]:
train_and_evaluate_model("MobileNet", MobileNet)


Training MobileNet...
17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 79s 694ms/step - accuracy: 0.5963 - loss: 0.9583 - val_accuracy: 0.8500 - val_loss: 0.4638 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 650ms/step - accuracy: 0.9374 - loss: 0.2483 - val_accuracy: 0.8855 - val_loss: 0.3344 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 651ms/step - accuracy: 0.9418 - loss: 0.1721 - val_accuracy: 0.9200 - val_loss: 0.2441 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 651ms/step - accuracy: 0.9634 - loss: 0.1200 - val_accuracy: 0.9100 - val_loss: 0.2565 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 642ms/step - accuracy: 0.9673 - loss: 0.1114 - val_accuracy: 0.9095 - val_loss: 0.2432 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 274ms/step - accuracy: 0.8456 - loss: 0.4264

Model: MobileNet
Training Accuracy: 97.00
Validation Ac

<Sequential name=sequential_5, built=True>

In [8]:
train_and_evaluate_model("MobileNetV2", MobileNetV2)


Training MobileNetV2...
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 83s 709ms/step - accuracy: 0.5785 - loss: 1.0074 - val_accuracy: 0.8570 - val_loss: 0.4140 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 651ms/step - accuracy: 0.9077 - loss: 0.3043 - val_accuracy: 0.9125 - val_loss: 0.2737 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 654ms/step - accuracy: 0.9417 - loss: 0.1956 - val_accuracy: 0.9190 - val_loss: 0.2307 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 649ms/step - accuracy: 0.9455 - loss: 0.1660 - val_accuracy: 0.9295 - val_loss: 0.2018 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 641ms/step - accuracy: 0.9649 - loss: 0.1236 - val_accuracy: 0.8955 - val_loss: 0.2560 - learning_rate: 1.0000e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 656ms/step - accuracy: 0.9551 - loss: 0.1296 - val_accuracy: 0.9325 - val_loss: 0.1802 

<Sequential name=sequential_6, built=True>

In [9]:
train_and_evaluate_model("EfficientNetB0", EfficientNetB0)


Training EfficientNetB0...
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 99s 756ms/step - accuracy: 0.2440 - loss: 1.3887 - val_accuracy: 0.2500 - val_loss: 1.3867 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 652ms/step - accuracy: 0.2424 - loss: 1.3877 - val_accuracy: 0.2500 - val_loss: 1.3864 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 648ms/step - accuracy: 0.2492 - loss: 1.3879 - val_accuracy: 0.2500 - val_loss: 1.3869 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 654ms/step - accuracy: 0.2377 - loss: 1.3869 - val_accuracy: 0.2500 - val_loss: 1.3865 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 651ms/step - accuracy: 0.2432 - loss: 1.3871 - val_accuracy: 0.2500 - val_loss: 1.3862 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.2642 - loss: 1.3839

Model: EfficientNetB0
Training Accuracy: 25.47
Val

<Sequential name=sequential_7, built=True>

In [10]:
train_and_evaluate_model("EfficientNetB7", EfficientNetB7)


Training EfficientNetB7...
258076736/258076736 ━━━━━━━━━━━━━━━━━━━━ 7s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 188s 1s/step - accuracy: 0.2596 - loss: 1.3959 - val_accuracy: 0.2500 - val_loss: 1.3865 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 695ms/step - accuracy: 0.2355 - loss: 1.3959 - val_accuracy: 0.2500 - val_loss: 1.3877 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 702ms/step - accuracy: 0.2438 - loss: 1.3952 - val_accuracy: 0.2500 - val_loss: 1.3868 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 697ms/step - accuracy: 0.2573 - loss: 1.3893 - val_accuracy: 0.2500 - val_loss: 1.3880 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 695ms/step - accuracy: 0.2428 - loss: 1.3896 - val_accuracy: 0.2500 - val_loss: 1.3853 - learning_rate: 5.0000e-05
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 271ms/step - accuracy: 0.2629 - loss: 1.3839

Model: EfficientNetB7
Training Accuracy: 26.06
Val

<Sequential name=sequential_8, built=True>

In [11]:
train_and_evaluate_model("NASNetMobile", NASNetMobile)


Training NASNetMobile...
19993432/19993432 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 127s 895ms/step - accuracy: 0.4796 - loss: 1.2214 - val_accuracy: 0.7430 - val_loss: 0.7298 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 658ms/step - accuracy: 0.8483 - loss: 0.4980 - val_accuracy: 0.7760 - val_loss: 0.6048 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 669ms/step - accuracy: 0.9107 - loss: 0.3142 - val_accuracy: 0.7955 - val_loss: 0.5328 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 669ms/step - accuracy: 0.9091 - loss: 0.2767 - val_accuracy: 0.8495 - val_loss: 0.4179 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 675ms/step - accuracy: 0.9232 - loss: 0.2328 - val_accuracy: 0.8385 - val_loss: 0.4357 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 275ms/step - accuracy: 0.7497 - loss: 0.7006

Model: NASNetMobile
Training Accuracy: 92.78
Valida

<Sequential name=sequential_9, built=True>

In [12]:
train_and_evaluate_model("EfficientNetV2B0", EfficientNetV2B0)


Training EfficientNetV2B0...
24274472/24274472 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 107s 786ms/step - accuracy: 0.2355 - loss: 1.3964 - val_accuracy: 0.2500 - val_loss: 1.3878 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 654ms/step - accuracy: 0.2616 - loss: 1.3890 - val_accuracy: 0.2500 - val_loss: 1.3906 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 650ms/step - accuracy: 0.2658 - loss: 1.3913 - val_accuracy: 0.2500 - val_loss: 1.3893 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 650ms/step - accuracy: 0.2702 - loss: 1.3870 - val_accuracy: 0.2500 - val_loss: 1.3991 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 656ms/step - accuracy: 0.2455 - loss: 1.3972 - val_accuracy: 0.2500 - val_loss: 1.3873 - learning_rate: 5.0000e-05
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 270ms/step - accuracy: 0.2433 - loss: 1.3877

Model: EfficientNetV2B0
Training Accuracy: 26.0

<Sequential name=sequential_10, built=True>

In [13]:
train_and_evaluate_model("ResNet50V2", ResNet50V2)


Training ResNet50V2...
94668760/94668760 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 90s 738ms/step - accuracy: 0.6639 - loss: 0.8871 - val_accuracy: 0.8850 - val_loss: 0.3361 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 653ms/step - accuracy: 0.9258 - loss: 0.2525 - val_accuracy: 0.9010 - val_loss: 0.2631 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 650ms/step - accuracy: 0.9362 - loss: 0.1914 - val_accuracy: 0.9340 - val_loss: 0.1800 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 665ms/step - accuracy: 0.9488 - loss: 0.1520 - val_accuracy: 0.9310 - val_loss: 0.1875 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 659ms/step - accuracy: 0.9601 - loss: 0.1286 - val_accuracy: 0.9430 - val_loss: 0.1574 - learning_rate: 1.0000e-04
Epoch 6/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 679ms/step - accuracy: 0.9698 - loss: 0.1045 - val_accuracy: 0.9480 - val_loss: 0.1431

<Sequential name=sequential_11, built=True>

In [14]:
train_and_evaluate_model("VGG19", VGG19)


Training VGG19...
80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 76s 698ms/step - accuracy: 0.3464 - loss: 1.3974 - val_accuracy: 0.3885 - val_loss: 1.3037 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 673ms/step - accuracy: 0.6303 - loss: 1.2155 - val_accuracy: 0.4035 - val_loss: 1.2406 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 687ms/step - accuracy: 0.6881 - loss: 1.1003 - val_accuracy: 0.4080 - val_loss: 1.1973 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 667ms/step - accuracy: 0.7263 - loss: 0.9879 - val_accuracy: 0.4145 - val_loss: 1.1643 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 661ms/step - accuracy: 0.7399 - loss: 0.9037 - val_accuracy: 0.4245 - val_loss: 1.1541 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 6s 247ms/step - accuracy: 0.3849 - loss: 1.3010

Model: VGG19
Training Accuracy: 74.28
Validation Accuracy: 

<Sequential name=sequential_12, built=True>

In [15]:
train_and_evaluate_model("InceptionResNetV2", InceptionResNetV2)


Training InceptionResNetV2...
219055592/219055592 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 120s 866ms/step - accuracy: 0.4520 - loss: 1.2373 - val_accuracy: 0.7950 - val_loss: 0.5437 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 671ms/step - accuracy: 0.8664 - loss: 0.4220 - val_accuracy: 0.8180 - val_loss: 0.4479 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 693ms/step - accuracy: 0.9073 - loss: 0.2859 - val_accuracy: 0.8230 - val_loss: 0.4460 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 677ms/step - accuracy: 0.9185 - loss: 0.2465 - val_accuracy: 0.8625 - val_loss: 0.3439 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 676ms/step - accuracy: 0.9356 - loss: 0.2028 - val_accuracy: 0.8860 - val_loss: 0.3025 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 264ms/step - accuracy: 0.8031 - loss: 0.5385

Model: InceptionResNetV2
Training Accuracy: 

<Sequential name=sequential_13, built=True>

In [16]:
train_and_evaluate_model("NASNetLarge", NASNetLarge)


Training NASNetLarge...
343610240/343610240 ━━━━━━━━━━━━━━━━━━━━ 9s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 161s 1s/step - accuracy: 0.6814 - loss: 0.8347 - val_accuracy: 0.8470 - val_loss: 0.4564 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 81s 766ms/step - accuracy: 0.9170 - loss: 0.2576 - val_accuracy: 0.8760 - val_loss: 0.3557 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 81s 766ms/step - accuracy: 0.9413 - loss: 0.1949 - val_accuracy: 0.8970 - val_loss: 0.3169 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 83s 789ms/step - accuracy: 0.9593 - loss: 0.1491 - val_accuracy: 0.8155 - val_loss: 0.4277 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 81s 769ms/step - accuracy: 0.9565 - loss: 0.1483 - val_accuracy: 0.8540 - val_loss: 0.3510 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 9s 356ms/step - accuracy: 0.8309 - loss: 0.4674

Model: NASNetLarge
Training Accuracy: 95.16
Validatio

<Sequential name=sequential_14, built=True>

In [17]:
train_and_evaluate_model("EfficientNetV2S", EfficientNetV2S)


Training EfficientNetV2S...
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 139s 857ms/step - accuracy: 0.3348 - loss: 1.3340 - val_accuracy: 0.4890 - val_loss: 1.1593 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 643ms/step - accuracy: 0.5125 - loss: 1.1506 - val_accuracy: 0.5360 - val_loss: 1.0640 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 666ms/step - accuracy: 0.5189 - loss: 1.0812 - val_accuracy: 0.5495 - val_loss: 0.9985 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 672ms/step - accuracy: 0.5534 - loss: 1.0371 - val_accuracy: 0.5765 - val_loss: 0.9583 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 666ms/step - accuracy: 0.5514 - loss: 1.0101 - val_accuracy: 0.6025 - val_loss: 0.9132 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 254ms/step - accuracy: 0.5357 - loss: 1.1368

Model: EfficientNetV2S
Training Accuracy: 56.44


<Sequential name=sequential_15, built=True>

In [18]:
train_and_evaluate_model("EfficientNetV2M", EfficientNetV2M)


Training EfficientNetV2M...
214201816/214201816 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 170s 961ms/step - accuracy: 0.2336 - loss: 1.3873 - val_accuracy: 0.2820 - val_loss: 1.3767 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 683ms/step - accuracy: 0.2813 - loss: 1.3822 - val_accuracy: 0.3035 - val_loss: 1.3708 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 694ms/step - accuracy: 0.2892 - loss: 1.3826 - val_accuracy: 0.3790 - val_loss: 1.3745 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 684ms/step - accuracy: 0.3004 - loss: 1.3728 - val_accuracy: 0.3290 - val_loss: 1.3716 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 671ms/step - accuracy: 0.3165 - loss: 1.3680 - val_accuracy: 0.4010 - val_loss: 1.3736 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 263ms/step - accuracy: 0.2927 - loss: 1.3779

Model: EfficientNetV2M
Training Accuracy: 31.0

<Sequential name=sequential_16, built=True>

In [19]:
train_and_evaluate_model("EfficientNetV2L", EfficientNetV2L)


Training EfficientNetV2L...
473176280/473176280 ━━━━━━━━━━━━━━━━━━━━ 12s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 213s 1s/step - accuracy: 0.2616 - loss: 1.3854 - val_accuracy: 0.4380 - val_loss: 1.3411 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 79s 751ms/step - accuracy: 0.2834 - loss: 1.3759 - val_accuracy: 0.5060 - val_loss: 1.3130 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 79s 747ms/step - accuracy: 0.3345 - loss: 1.3646 - val_accuracy: 0.3255 - val_loss: 1.3061 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 78s 746ms/step - accuracy: 0.3461 - loss: 1.3584 - val_accuracy: 0.4670 - val_loss: 1.2837 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 78s 746ms/step - accuracy: 0.3434 - loss: 1.3496 - val_accuracy: 0.5000 - val_loss: 1.2530 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 8s 327ms/step - accuracy: 0.4409 - loss: 1.3368

Model: EfficientNetV2L
Training Accuracy: 34.69


<Sequential name=sequential_17, built=True>

In [20]:
train_and_evaluate_model("MobileNetV3Large", MobileNetV3Large)


Training MobileNetV3Large...
12683000/12683000 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 91s 754ms/step - accuracy: 0.2592 - loss: 1.4267 - val_accuracy: 0.2725 - val_loss: 1.3793 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 662ms/step - accuracy: 0.2766 - loss: 1.3799 - val_accuracy: 0.2850 - val_loss: 1.3696 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 665ms/step - accuracy: 0.3074 - loss: 1.3736 - val_accuracy: 0.2545 - val_loss: 1.3704 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 666ms/step - accuracy: 0.3026 - loss: 1.3688 - val_accuracy: 0.4090 - val_loss: 1.3531 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 660ms/step - accuracy: 0.3272 - loss: 1.3620 - val_accuracy: 0.3915 - val_loss: 1.3485 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 266ms/step - accuracy: 0.2713 - loss: 1.3796

Model: MobileNetV3Large
Training Accuracy: 36.72

<Sequential name=sequential_18, built=True>

In [21]:
train_and_evaluate_model("MobileNetV3Small",MobileNetV3Small)


Training MobileNetV3Small...
4334752/4334752 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 91s 764ms/step - accuracy: 0.2538 - loss: 1.4181 - val_accuracy: 0.2165 - val_loss: 1.3825 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 650ms/step - accuracy: 0.2900 - loss: 1.3809 - val_accuracy: 0.3355 - val_loss: 1.3816 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 69s 650ms/step - accuracy: 0.2756 - loss: 1.3799 - val_accuracy: 0.3115 - val_loss: 1.3837 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 68s 648ms/step - accuracy: 0.3015 - loss: 1.3776 - val_accuracy: 0.2425 - val_loss: 1.3800 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 67s 637ms/step - accuracy: 0.3269 - loss: 1.3749 - val_accuracy: 0.3330 - val_loss: 1.3779 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 257ms/step - accuracy: 0.2403 - loss: 1.3818

Model: MobileNetV3Small
Training Accuracy: 32.50
V

<Sequential name=sequential_19, built=True>

In [22]:
train_and_evaluate_model("ResNet101", ResNet101)


Training ResNet101...
171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 98s 775ms/step - accuracy: 0.2708 - loss: 1.4059 - val_accuracy: 0.2545 - val_loss: 1.3657 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 70s 669ms/step - accuracy: 0.3442 - loss: 1.3619 - val_accuracy: 0.3935 - val_loss: 1.3516 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 682ms/step - accuracy: 0.3556 - loss: 1.3500 - val_accuracy: 0.2900 - val_loss: 1.3413 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 682ms/step - accuracy: 0.3709 - loss: 1.3290 - val_accuracy: 0.3675 - val_loss: 1.3262 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 679ms/step - accuracy: 0.4164 - loss: 1.3163 - val_accuracy: 0.4605 - val_loss: 1.3109 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 270ms/step - accuracy: 0.2802 - loss: 1.3610

Model: ResNet101
Training Accuracy: 43.47
Validation 

<Sequential name=sequential_20, built=True>

In [23]:
train_and_evaluate_model("ResNet152", ResNet152)


Training ResNet152...
234698864/234698864 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 120s 873ms/step - accuracy: 0.2851 - loss: 1.3956 - val_accuracy: 0.3685 - val_loss: 1.3628 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 75s 713ms/step - accuracy: 0.3086 - loss: 1.3666 - val_accuracy: 0.2550 - val_loss: 1.3594 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 694ms/step - accuracy: 0.3310 - loss: 1.3533 - val_accuracy: 0.4430 - val_loss: 1.3265 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 76s 716ms/step - accuracy: 0.3946 - loss: 1.3395 - val_accuracy: 0.3690 - val_loss: 1.3124 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 74s 706ms/step - accuracy: 0.3506 - loss: 1.3302 - val_accuracy: 0.4015 - val_loss: 1.2989 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 279ms/step - accuracy: 0.3999 - loss: 1.3600

Model: ResNet152
Training Accuracy: 39.84
Validation

<Sequential name=sequential_21, built=True>

In [24]:
train_and_evaluate_model("ResNet101V2", ResNet101V2)


Training ResNet101V2...
171317808/171317808 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 95s 765ms/step - accuracy: 0.5827 - loss: 1.0283 - val_accuracy: 0.8645 - val_loss: 0.3910 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 680ms/step - accuracy: 0.9125 - loss: 0.2744 - val_accuracy: 0.9135 - val_loss: 0.2699 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 678ms/step - accuracy: 0.9429 - loss: 0.1894 - val_accuracy: 0.9280 - val_loss: 0.2132 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 676ms/step - accuracy: 0.9635 - loss: 0.1311 - val_accuracy: 0.9390 - val_loss: 0.1711 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 71s 666ms/step - accuracy: 0.9650 - loss: 0.1148 - val_accuracy: 0.9475 - val_loss: 0.1538 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 268ms/step - accuracy: 0.8796 - loss: 0.3424

Model: ResNet101V2
Training Accuracy: 96.53
Validat

<Sequential name=sequential_22, built=True>

In [25]:
train_and_evaluate_model("ResNet152V2", ResNet152V2)


Training ResNet152V2...
234545216/234545216 ━━━━━━━━━━━━━━━━━━━━ 6s 0us/step
Epoch 1/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 109s 821ms/step - accuracy: 0.5949 - loss: 0.9684 - val_accuracy: 0.8805 - val_loss: 0.3497 - learning_rate: 1.0000e-04
Epoch 2/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 688ms/step - accuracy: 0.9179 - loss: 0.2807 - val_accuracy: 0.9225 - val_loss: 0.2309 - learning_rate: 1.0000e-04
Epoch 3/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 690ms/step - accuracy: 0.9436 - loss: 0.1819 - val_accuracy: 0.9390 - val_loss: 0.1929 - learning_rate: 1.0000e-04
Epoch 4/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 73s 684ms/step - accuracy: 0.9574 - loss: 0.1477 - val_accuracy: 0.9330 - val_loss: 0.1987 - learning_rate: 1.0000e-04
Epoch 5/10
100/100 ━━━━━━━━━━━━━━━━━━━━ 72s 682ms/step - accuracy: 0.9519 - loss: 0.1378 - val_accuracy: 0.9555 - val_loss: 0.1404 - learning_rate: 1.0000e-04
25/25 ━━━━━━━━━━━━━━━━━━━━ 7s 275ms/step - accuracy: 0.8735 - loss: 0.3293

Model: ResNet152V2
Training Accuracy: 95.50
Valida

<Sequential name=sequential_23, built=True>